# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
g_key = key_dict['maps']
gmaps.configure(api_key=g_key)

# Output File (CSV)
output_data_file = "../output_data/cities.csv"


NameError: name 'key_dict' is not defined

In [44]:
df=pd.read_csv(output_data_file)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../output_data/cities.csv'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
fig=gmaps.figure()
fig


Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
city_data.head()

,City,Id,country,lat,lng,Cloudiness,humidity,temperature,wind_speed


In [43]:
city_data["humidity"].max()

nan

In [29]:
location=city_data[["lat","lng"]].astype(float)
weight_humidity=city_data["humidity"].astype(float)

location=city_data[["lat","lng"]].astype(float)
city_humidity=city_data["humidity"].astype(float)

heat_layer=gmaps.heatmap_layer(location,weights=weight_humidity,dissipating=False,
                               max_intensity=100,point_radius=2)
fig.add_layer(heat_layer)
fig

TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
filt=((city_data["temperature"]>70)&
     (city_data["temperature"]<80)&
     (city_data["wind_speed"]<10)&
     (city_data["Cloudiness"]==0))

city_data=city_data.loc[filt,:]
city_data.dropna()
        
city_data

,City,Id,country,lat,lng,Cloudiness,humidity,temperature,wind_speed


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
hotel_df=city_data[["City","country","lat","lng"]]
hotel_df=hotel_df.reset_index(drop=True)
hotel_df["hotel name"]=""

hotel_df

,City,country,lat,lng,hotel name


In [32]:
for index,row in hotel_df.iterrows():
    try:
        lat=row["lat"]
        lng=row["lng"]
        location=f"{lat},{lng}"
        base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        parameters={"location":location,"radius":5000,"keyword":"Hotel","key":g_key}
        repo=requests.get(base_url,params=parameters).json()
    
        hotel_df.loc[index,"hotel name"]=repo['results'][0]["name"]
        
    except IndexError:
        
        print("data not found")
            

In [33]:
hotel_df

,City,country,lat,lng,hotel name


In [34]:
hotel_df["hotel name"].replace("",np.nan,inplace=True)

hotel_df.dropna(inplace=True)
hotel_df.rename(columns={"country":"Country",
                        "hotel name":"Hotel Name"},inplace=True)
hotel_df

,City,Country,lat,lng,Hotel Name


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [36]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(markers)
fig.add_layer(heat_layer)


# Display figure
fig

NameError: name 'locations' is not defined